In [17]:
import cv2
import mediapipe as mp
import time
import math
import sys
import numpy as np

class handDetector():
    def __init__(self, mode=False, maxHands=1, modelComplexity=1, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.modelComplexity = modelComplexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.modelComplexity, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        #print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                #print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                xList.append(cx)
                yList.append(cy)
                #print(id, cx, cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255,0,255), cv2.FILLED)
            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax
            bbox = list(bbox)
            a = xmax - xmin
            b = ymax - ymin
            if len(bbox) == 4:
                if a < b:
                    razn = (b - a) //2 
                    bbox[2] = bbox[2] + razn
                    bbox[0] = bbox[0] - razn
                else:
                    razn = (a - b) //2 
                    bbox[3] = bbox[3] + razn
                    bbox[1] = bbox[1] - razn
            #tempim = Image.open('0.png').convert('L')
            #rukaimg = img.crop((bbox[0], bbox[1], bbox[2], bbox[3]))
            #cv2.imshow("RUKA", rukaimg)
            #cv2.waitKey(1)
            #if cv2.waitKey(20) == 27: # exit on ESC
                #break
            #print(bbox)
            
                

            if draw:
                cv2.rectangle(img, (bbox[0]-20, bbox[1]-20), (bbox[2]+20, bbox[3]+20), (0, 255, 0), 2)
        return self.lmList, bbox

    def findDistance(self, p1, p2, img, draw=True):
        x1, y1 = self.lmList[p1][1], self.lmList[p1][2]
        x2, y2 = self.lmList[p2][1], self.lmList[p2][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2

        if draw:
            cv2.circle(img, (x1,y1), 15, (255,0,255), cv2.FILLED)
            cv2.circle(img, (x2,y2), 15, (255,0,255), cv2.FILLED)
            cv2.line(img, (x1,y1), (x2,y2), (255,0,255), 3)
            cv2.circle(img, (cx,cy), 15, (255,0,255), cv2.FILLED)

        length = math.hypot(x2-x1, y2-y1)
        return length, img, [x1, y1, x2, y2, cx, cy]

    def fingersUp(self):
        fingers = []

        # Thumb
        if self.lmList[self.tipIds[0]][1] < self.lmList[self.tipIds[0]-1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # 4 Fingers
        for id in range(1,5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        return fingers

In [24]:
stdoutOrigin=sys.stdout 
sys.stdout = open("log.txt", "w") #ДВЕ СТРОЧКИ ОТВЕЧАЮТ ЗА НАЧАЛО ВВОДА В log.txt
data = []

pTime = 0
cTime = 0
cap = cv2.VideoCapture(0)
detector = handDetector()
i = 0
while True:
    if i > 300:
        break
    i += 1
    success, img = cap.read()
    img = cv2.flip(img,1) # Mirror flip
    save = img
    img = detector.findHands(img)
    lmList = detector.findPosition(img)
    
    #if len(lmList) != 0:
        #print(lmList[1])
    cTime = time.time()
    fps = 1. / (cTime - pTime)
    pTime = cTime
    if len(lmList) != 0:
        if len(lmList[1]) == 4:
            crop_img = img[lmList[1][1] - 20: lmList[1][3] + 20, lmList[1][0] - 20 :lmList[1][2] + 20]
            #cv2.putText(crop_img, str(int(fps)), (10,70), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)
            try:
                up_width = 100
                up_height = 100
                up_points = (up_width, up_height)
                resized_up = cv2.resize(crop_img, up_points, interpolation= cv2.INTER_LINEAR)
                data.append(resized_up)
                cv2.imshow("Image", resized_up)
                cv2.waitKey(1)
                
            except:
                nothing = 1
    if cv2.waitKey(20) == 27: # exit on ESC
        break

cv2.destroyWindow("Image")
cap.release()
cv2.waitKey(1)
data = np.array(data)
np.set_printoptions(threshold=sys.maxsize)
print(data)

sys.stdout.close()
sys.stdout=stdoutOrigin #ДВЕ СТРОЧКИ ЗА ЗАКРЫТИЕ log.txt